# Semantic Kernel

In this code sample, you will use the [**Semantic Kernel**](https://aka.ms/ai-agents-beginners/semantic-kernel) **AI** framework to create a basic agent.
The goal of this sample is to show you the steps that we will later use in the additional code samples when implementing the different agentic patterns.

## Prepare the project

Import necessary packages:

In [1]:
import os

Import necessary entities:

In [2]:
from random import choice
from typing import Annotated
from openai import AsyncOpenAI
from dotenv import load_dotenv
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

Load enviroment for using a system variables:

In [3]:
load_dotenv();

## Prepare agent functionality

In this sample, we will use [**GitHub** models](https://aka.ms/ai-agents-beginners/github-models) for access to the **LLM**.
The `ai_model_id` is defined as `gpt-4o-mini`.
Try changing the model to another model available on the **GitHub** models marketplace to see the different results.
For us to use the **Azure Inference SDK** that is used for the `base_url` for **GitHub** models, we will use the `OpenAIChatCompletion` connector within **Semantic Kernel**.
There are also other [available connectors](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) to use **Semantic Kernel** for other model providers.

Prepare a list of vacation destinations:

In [4]:
vacation_destinations: list[str] = [
    "Tokyo, Japan",
    "Cairo, Egypt",
    "Paris, France",
    "New York, USA",
    "Berlin, Germany",
    "Bali, Indonesia",
    "Barcelona, Spain",
    "Sydney, Australia",
    "Rio de Janeiro, Brazil",
    "Cape Town, South Africa",
]

Create a plugin class:

In [5]:
class DestinationsPlugin:
    """
    A plugin that manages and provides vacation destinations.

    :Attributes:
        last_destination (str | None): The last destination that was returned.
                                       Default: None.
        vacation_destinations (list[str]): The list of available destinations.
    """

    def __init__(self, vacation_destinations: list[str]) -> None:
        """
        Initializes the `DestinationsPlugin` with a list of destinations.

        :Parameters:
            vacation_destinations (list[str]): The list of available
                                               destinations.
        """

        self.last_destination: str | None = None
        self.vacation_destinations: list[str] = vacation_destinations

    @kernel_function(description="Returns a random vacation destination.")
    def get_random_vacation_destination(
        self
    ) -> Annotated[str, "Returns a random vacation destination."]:
        """
        Selects and returns a random vacation destination from the available
        options.

        :Returns:
            str: A randomly selected vacation destination.
        """

        available_destinations: list[str] = self.vacation_destinations.copy()

        if self.last_destination and (len(available_destinations, ) > 1):
            available_destinations.remove(self.last_destination, )

        self.last_destination = choice(available_destinations, )

        return self.last_destination

## Create a client

Create a client instance:

In [6]:
client: AsyncOpenAI = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN", ),
    base_url="https://models.inference.ai.azure.com/",
)

Create an **AI** service that will be used by the `ChatCompletionAgent`:

In [7]:
chat_completion_service: OpenAIChatCompletion = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Creating the agent

Below we create the agent called `TravelAgent`.
For this example, we are using very simple instructions.
You can change these instructions to see how the agent responds differently.

In [8]:
agent: ChatCompletionAgent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[
        DestinationsPlugin(vacation_destinations=vacation_destinations, ),
    ],
    name="TravelAgent",
    instructions="You are a helpful AI agent that can help plan vacations for" +
                 " customers at random destinations.",
)

## Running the agent

Now we can run the agent by defining a thread of type `ChatHistoryAgentThread`.
Any required system messages are provided to the agent's `invoke_stream` `messages` keyword argument.
After these are defined, we create a `user_inputs` that will be what the user is sending to the agent.
In this case, we have set this message to `Plan me a sunny vacation.`.
Feel free to change this message to see how the agent responds differently.

In [9]:
async def main() -> None:
    """
    Executes a streaming conversation with an AI agent by processing multiple
    user inputs sequentially.
    """

    thread: ChatHistoryAgentThread | None = None
    user_inputs: list[str] = [
        "Plan me a day trip.",
        "Give me a random place for vacation",
    ]

    for user_input in user_inputs:
        print(f"\nUser message:\n{user_input}\n", )

        first_chunk: bool = True

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            if first_chunk:
                print(
                    f"{response.name}:\n",
                    end="",
                    flush=True,
                )

                first_chunk = False

            print(
                f"{response}",
                end="",
                flush=True,
            )

            thread = response.thread

        print()

    await thread.delete() if thread else None

Launch the agent workflow:

In [10]:
await main()


User message:
Plan me a day trip.

TravelAgent:
How about a day trip to New York City? Here's a suggested itinerary:

### Morning:
- **8:00 AM:** Start your day with breakfast at a classic New York diner. Try places like "Eisenberg's Sandwich Shop" for a real New York experience.
- **9:00 AM:** Visit the **Statue of Liberty**. Take a ferry from Battery Park. Enjoy the views of the skyline as you approach Liberty Island.

### Afternoon:
- **12:00 PM:** Head back to Manhattan and grab lunch in the Financial District. Consider a stop at "Fraunces Tavern," a historic site with delicious food.
- **1:30 PM:** Explore **5th Avenue** and do some shopping. Visit iconic stores like Saks Fifth Avenue or the Apple Store.
- **3:00 PM:** Take a stroll through **Central Park**. Rent a bike or simply walk the paths while enjoying the scenery.

### Evening:
- **5:00 PM:** Visit the **Metropolitan Museum of Art** or the **American Museum of Natural History**. Both are magnificent and offer a break from